<!-- NOTEBOOK_METADATA source: "⚠️ Jupyter Notebook" title: "Observability for BeeAI Framework with Langfuse Integration" sidebarTitle: "BeeAI" logo: "/images/integrations/beeai_icon.svg" description: "Learn how to integrate Langfuse with the BeeAI Framework for comprehensive tracing and debugging of your AI agent applications." category: "Integrations" -->

# Trace BeeAI Framework Applications in Langfuse

This notebook shows how to trace and observe BeeAI Framework applications with Langfuse using OpenTelemetry instrumentation.

> **What is BeeAI Framework?** [BeeAI Framework](https://framework.beeai.dev) is a comprehensive toolkit for building intelligent, autonomous agents and multi-agent systems. It provides everything you need to create agents that can reason, take actions, and collaborate to solve complex problems in both Python and TypeScript.

> **What is the BeeAI Framework Python SDK?** The [BeeAI Framework Python SDK](https://github.com/i-am-bee/beeai-framework) is a production-ready library that provides core building blocks like agents, workflows, backend connections, tools, memory management, and observability features for building intelligent AI applications.

> **What is Langfuse?** [Langfuse](https://langfuse.com) is an open source platform for LLM observability and monitoring. It helps you trace and monitor your AI applications by capturing metadata, prompt details, token usage, latency, and more.


<!-- STEPS_START -->
## Step 1: Install Dependencies

Before you begin, install the necessary packages in your Python environment:


In [ ]:
%pip install beeai-framework langfuse openinference-instrumentation-beeai opentelemetry-sdk opentelemetry-exporter-otlp

## Step 2: Configure Langfuse SDK

Next, set up your Langfuse API keys. You can get these keys by signing up for a free [Langfuse Cloud](https://cloud.langfuse.com/) account or by [self-hosting Langfuse](https://langfuse.com/self-hosting). These environment variables are essential for the Langfuse client to authenticate and send data to your Langfuse project.

You will also need to configure your LLM provider credentials. BeeAI Framework supports multiple providers including OpenAI, Ollama, watsonx.ai, and others.

In [ ]:
import os

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..." 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." 
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

# Your LLM provider API key (example with OpenAI, adjust for your provider)
os.environ["OPENAI_API_KEY"] = "sk-..."  
# For Ollama (local), no API key needed
# For other providers, set appropriate environment variables

With the environment variables set, we can now initialize the Langfuse client. `get_client()` initializes the Langfuse client using the credentials provided in the environment variables.

In [2]:
from langfuse import get_client

# Initialise Langfuse client and verify connectivity
langfuse = get_client()
assert langfuse.auth_check(), "Langfuse auth failed - check your keys ✋"

## Step 3: OpenTelemetry Instrumentation

Use the [`BeeAIInstrumentor`](https://github.com/Arize-ai/openinference/tree/main/python/instrumentation/openinference-instrumentation-beeai) library to wrap [BeeAI Framework](https://framework.beeai.dev) calls and send OpenTelemetry spans to Langfuse.

In [3]:
from openinference.instrumentation.beeai import BeeAIInstrumentor

BeeAIInstrumentor().instrument()

## Step 4: Run an Example

In [ ]:
import asyncio
from beeai_framework.agents.react import ReActAgent
from beeai_framework.agents.types import AgentExecutionConfig
from beeai_framework.backend.chat import ChatModel
from beeai_framework.backend.types import ChatModelParameters
from beeai_framework.memory import TokenMemory
from beeai_framework.tools.search import WikipediaTool
from beeai_framework.tools.weather.openmeteo import OpenMeteoTool

# Initialize the language model
llm = ChatModel.from_name(
    "openai:gpt-4o-mini",  # or "ollama:granite3.3:8b" for local Ollama
    ChatModelParameters(temperature=0.7),
)

# Create tools for the agent
tools = [
    WikipediaTool(),
    OpenMeteoTool(),
]

# Create a ReAct agent with memory
agent = ReActAgent(
    llm=llm, 
    tools=tools, 
    memory=TokenMemory(llm)
)

# Run the agent
async def main():
    response = await agent.run(
        prompt="What is Langfuse and what's the current weather in San Francisco?",
        execution=AgentExecutionConfig(
            max_retries_per_step=3, 
            total_max_retries=10, 
            max_iterations=5
        ),
    )
    print("Agent Response:", response.result.text)
    return response

# Run the example
response = await main()

In [ ]:
# Multi-Agent Workflow Example
from beeai_framework.workflows.agent import AgentWorkflow, AgentWorkflowInput

# Create a multi-agent workflow
workflow = AgentWorkflow(name="Research Assistant")

# Add specialized agents to the workflow
workflow.add_agent(
    name="Researcher",
    role="A diligent researcher.",
    instructions="You look up and provide information about specific topics using available tools.",
    tools=[WikipediaTool()],
    llm=llm,
)

workflow.add_agent(
    name="WeatherAnalyst",
    role="A weather reporting specialist.",
    instructions="You provide detailed weather reports and analysis.",
    tools=[OpenMeteoTool()],
    llm=llm,
)

workflow.add_agent(
    name="Synthesizer",
    role="A data synthesis expert.",
    instructions="You combine information from different sources into coherent summaries.",
    llm=llm,
)

# Run the workflow
async def run_workflow():
    response = await workflow.run(
        inputs=[
            AgentWorkflowInput(
                prompt="Research what Langfuse is and its main features.",
            ),
            AgentWorkflowInput(
                prompt="Get the current weather conditions in Paris, France.",
                expected_output="Temperature, conditions, and any notable weather information.",
            ),
            AgentWorkflowInput(
                prompt="Combine the Langfuse research and Paris weather into a brief summary.",
                expected_output="A paragraph combining both pieces of information.",
            ),
        ]
    ).on(
        "success",
        lambda data, event: print(
            f"\n-> Step '{data.step}' completed:\n{data.state.final_answer}\n"
        ),
    )
    
    print("==== Final Workflow Result ====")
    print(response.result.final_answer)
    return response

# Run the workflow example
workflow_response = await run_workflow()

### View Traces in Langfuse

After executing the application, navigate to your Langfuse Trace Table. You will find detailed traces of the application's execution, providing insights into the agent conversations, tool calls, LLM interactions, inputs, outputs, and performance metrics.

![Langfuse Trace](https://langfuse.com/images/cookbook/integration_beeai/beeai-trace.png)

[View trace in Langfuse](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/beeai-example-trace?timestamp=2025-01-15T10%3A30%3A00.000Z&display=details)

<!-- STEPS_END -->

<!-- MARKDOWN_COMPONENT name: "LearnMore" path: "@/components-mdx/integration-learn-more.mdx" -->